In [ ]:
import numpy as np 
import pandas as pd # for reading csv file

#train and test images are .7z archives, we need to unpack them
!pip install py7zr

from py7zr import unpack_7zarchive
import shutil #sh utill

#Before using, we need to register unpack format
shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)

#unpack train images in /kaggle/working or /kaggle/temp
shutil.unpack_archive('/kaggle/input/cifar-10/train.7z', '/kaggle/temp/')#this temp directory is not visible to use explicitly
#here we are unzipping the files to "/kaggle/temp/" directory

#trainLabels.csv file have lebel of training data


In [ ]:
#just checking the filenames
import os
i=0
for root, dirnames,fnames in os.walk("/kaggle/temp/train"):
    for fname in sorted(fnames):
        print(fname)
        print(os.path.join(root,fname))
        i=i+1
        if(i==50):
            break

In [ ]:
import torch
if torch.cuda.is_available():
    device=torch.device(type="cuda", index=0)
else:
    device=torch.device(type="cpu", index=0) 
    
    
print(device)

In [ ]:
#first fetching the class names from trainLabels.csv

train_labels=pd.read_csv("/kaggle/input/cifar-10/trainLabels.csv", header='infer')

#unique labels
classes=train_labels['label'].unique() # so we have total 50k images now to find all the unique classes 
#we can apply unique() on "label" so classes will be all the 10 unique classes that this dataset have

#confirming
print(classes)

#classnames to classids
name2num={} #whenever we want to use numeric representation of classes then we can use this dictionary
i=0
for name in classes:
    name2num[name]=i
    i=i+1

num2name={}#now in code we will be working with numerical values of class but at the time of submission 
#we have to submit classes name so we can use this dictionary at that time
for i in range(len(classes)): #from 0 to 9
    num2name[i]=classes[i]

In [ ]:
from torch.utils.data import Dataset #because we are writing custom dataset and this module will inherite
#the Dataset
from torch.utils.data import DataLoader#dataloader to wrape around custom dataset
import os #because my images are there in directory like /kaggle/temp/train now to walk over it we have
#to import os
from torchvision.io import read_image #it will allow to read .png and .jpeg images and returs a tensor
from torchvision.transforms import ToTensor, Normalize, Resize, Compose#normalize we will be using z-score
#normalization and compose for to apply group of transformations and resize is for like our image is of
#32X32 for pretrained network like imageNET and it uses 224X224

# normalization will work like we have 224X224X3 for all the 1M images so for all 1M images we have 3 channels
# so we will add all R values of 1M images (total 1M*224 R intensities) and divide by total R values which is 
# nothing but 1M*224 
# and same for G and B

# and from that we will be getting Rnorm=(Rvalue-(mean(R))/(SD(R))) so this normalization will be done by 
# above imported normalize


#now we are creating our custom dataset named as TrainDataset and any custom dataset have three method
#first is init, len, getitem
#init-> at the time of creating instance of this dataset to initialize all the values we will use it
#len will return how many entries available for this dataset
#getiem will return {image,lable} pair one at a time
class TrainDataset(Dataset):
    def __init__(self, imgpath, labelpath):#imgpath will be "/kaggle/temp/train" and labelpath is #"/kaggle/input/cifar-10/trainLabels.csv"
        super().__init__()#we are calling base class constructor
        self.imgpath=imgpath
        self.labelpath=labelpath
        self.labels=pd.read_csv(labelpath, header='infer') #since labelpath is .csv file and it contains 
        #two colums id and lable so without reading it's header we are storing id and lable (DF) into labels variable
        self.transform=Compose([Resize((224,224), antialias=True), Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
        #so as we have imported that compose now we can use it here like
        #to use it we have to provide list of all the transformation seperated by "," so all the transformation will
        #be applied in sequence
#         1)as we have discussed above to change 32X32 to 223X224 we are applying resize transformation 
#         2)so we are using z-score normalization so we need 3 means and 3 SDs (for R,G,B)

    def __len__(self):
        return self.labels.shape[0] #we have 50,000 * 2 so it will return 50,000
    
    def __getitem__(self,idx):# so here one by one we are providing idx for which we need image,lable pair
        finalpath=os.path.join(self.imgpath,str(idx+1))+'.png'  #here idx will be from [0,49,999] but images are
        #from 1.png to 50000.png so we are finding that image path here 
        # so for any exmaple if idx=0 then finalpath will be "/kaggle/temp/train/1.png"
        img=read_image(finalpath)/255 
        img=self.transform(img) #now we are applying that two transformer so from 32X32 it will resize to 224X224
        # and along with that all the R,G,B intensities will be normalized
        label=self.labels.iloc[idx,1] #now we have prepocessed image and we need it's label so self.labels
        # is dataframe so [idx,1] mean idx'th row and 1'st column because lable is on second column like labels is 
#         DF having two columns [id,label] so here we are getting label in name form like "cat","aeroplae" etc.
#and here we are not doing idx+1 because our header is inferd so for image1 label is at 0th row , for image2 label is at 1st row 
# and we are providing idx also in form of 0,1,2,......
        label=name2num[label]#we are converting this label into numeric form because for computation.
        return img,label #here img is tensor because read_image already convers image to tensor

traindataset=TrainDataset('/kaggle/temp/train','/kaggle/input/cifar-10/trainLabels.csv')        

batch_size=64    
traindataloader=DataLoader(dataset=traindataset, batch_size=batch_size)

In [ ]:
import torch.nn as nn
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
#here we are using one of the popular CNN mobilenet_v3_large and it's weight as MobileNet_V3_Large_Weights

#now we are creating NN and any NN have atleast two methods first is init and second is forward
#in init method we are initializing all the layers (they are not connected only initialized)
class Cifar10Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrainednet=mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT) #.DEFAULT mean import best 
        #available MobileNet_V3_Large_Weights so pretraindnet is now referring to mobilenet_v3_large
        #mobilenet have 3 childrens 1)features 2)avgpool and 3) classifier 
        #here at the last layer outfeatures are 1000 means it is trained for 1000 classes
        #so self.pretrainednet is entire network and have 3 chiles as above and self.pretrainednet.classifier
#         is the  last child and it has to be changed because it have total 1000 neuron but we only need 10
#         classes so it is totally mobilenet network but we are only changing the last layer 1000 -> 10 classes

        #means  self.pretrainednet have same features and avgpool but have different classifier and it is below
        self.pretrainednet.classifier=nn.Sequential(
            nn.Linear(in_features=960, out_features=1280, 
                   bias=True),nn.Hardswish(), 
            nn.Dropout(p=0.2, inplace=True), 
            nn.Linear(in_features=1280, out_features=10, 
                      bias=True)
        )
        #since outfeature is 10 so we will return 10 classed probability one for each
    
    
    def forward(self,x):#x will be 64 pairs of 224 X 224 X 3 image and 1 corrosponding lable
        x=self.pretrainednet(x) # so here x will become 64 X 10 because at last we have out_features as 10
        return x 

In [ ]:

def train_one_epoch(dataloader, model,loss_fn, optimizer):
    model.train()# we are telling model that now you will be in train mode 
    track_loss=0
    num_correct=0
    num_param=0
    
    for i, (imgs, labels) in enumerate(dataloader):
        imgs=imgs.to(device)
        labels=labels.to(device)
        pred=model(imgs)
                    
        loss=loss_fn(pred,labels)
        track_loss+=loss.item()
        num_correct+=(torch.argmax(pred,dim=1)==labels).type(torch.float).sum().item()
        
        running_loss=round(track_loss/(i+(imgs.shape[0]/batch_size)),2)
        running_acc=round((num_correct/((i*batch_size+imgs.shape[0])))*100,2)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i%100==0:
            print("Batch:", i+1, "/",len(dataloader), "Running Loss:",running_loss, "Running Accuracy:",running_acc)
            
    epoch_loss=running_loss
    epoch_acc=running_acc
    return epoch_loss, epoch_acc

avgpool do not have any wts and biases so there is not to do anything with it
#goal 
1) backpropogate only at classifier i.e. at the last layers of network means parametes of features not be changed so require_grad will be false here for features
2) backpropogate to entire network so here require grad of features will be true 


1)
for param in model.pretrainednet.features.parameters():
    param.requires_grad=True
    
 
 here model is is instance of cifar10Net and pretrainednet is reffering to mobile_net_v3 with changed classifier and it have three childs features, avgpool and classifier so model.pretrainednet.features have so many parameters  so one by one it will go in param variable 
 
 and param.requires_grad=False means these parameter will not be updated during backpropogation and it is only for features not for all the layers in network means we are only backpropogating into classifier that mean all the weights and baises of classifier will be changed accoring to earlier layers in network so we brought classifier's wts and biases to that level that earliear layers have
 
 
 2)
 for param in model.pretrainednet.features.parameters():
    param.requires_grad=True

loss_fn=nn.CrossEntropyLoss() #here we are using loss function as crossentropyloss
lr=0.001
#optimizer=torch.optim.SGD(params=model.parameters(), lr=lr)
optimizer=torch.optim.Adam(params=model.parameters(), lr=lr) #here parms will be all the parameters but we already specified that requires_grad =False for features so it will not be updated
n_epochs=30





In [ ]:
model=Cifar10Net()
model=model.to(device)

for param in model.pretrainednet.features.parameters():
    param.requires_grad=False

loss_fn=nn.CrossEntropyLoss()
lr=0.001
#optimizer=torch.optim.SGD(params=model.parameters(), lr=lr)
optimizer=torch.optim.Adam(params=model.parameters(), lr=lr)
n_epochs=30

for i in range(n_epochs):
    print("Epoch No:",i+1)
    train_epoch_loss, train_epoch_acc=train_one_epoch(traindataloader,model,loss_fn,optimizer)
    print("Training:", "Epoch Loss:", train_epoch_loss, "Epoch Accuracy:", train_epoch_acc)
    print("--------------------------------------------------")

for param in model.pretrainednet.features.parameters():
    param.requires_grad=True

for i in range(n_epochs):
    print("Epoch No:",i+1)
    train_epoch_loss, train_epoch_acc=train_one_epoch(traindataloader,model,loss_fn,optimizer)
    print("Training:", "Epoch Loss:", train_epoch_loss, "Epoch Accuracy:", train_epoch_acc)
    print("--------------------------------------------------")

In [ ]:
#unpacking test images, there are 3 lacs images. This will take some time
shutil.unpack_archive('/kaggle/input/cifar-10/test.7z', '/kaggle/temp/') # here length of test is 3lakhs
#so sampleSubmission.csv file will be having 3lakhs entries

#unregister unpack format, we are done with it
shutil.unregister_unpack_format('7zip')#, ['.7z'], unpack_7zarchive)

In [ ]:
class TestDataset(Dataset):
    def __init__(self, imgpath):#imgpath will be "kaggle/temp/test"
        super().__init__()
        self.imgpath=imgpath 
        _,_,self.files=next(os.walk(self.imgpath)) # next(os.walk(self.imgpath)) will return 3 things 
#         1) path to root directory i.e. kaggle/temp/test
#         2) list of all the subdirectories
#         3) list of all the files that it contains
#         so _,_,self.files means we are not interested into first two thing i.e. path to root directory and 
#         list of all the subdirectory we only need all the images i.e. 3rd thing so we are storing it into
#         files variable like files will have list of all the 3 lakhs images path
        self.length=len(self.files)# it will be 3lakhs
        self.transform=Compose([Resize((224,224), antialias=True), Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])        
        #we are providing same tranformer as traing 
    def __len__(self):
        return self.length
    
    def __getitem__(self,idx):
        finalpath=os.path.join(self.imgpath,str(idx+1))+'.png'
        img=read_image(finalpath)/255.0
        img=self.transform(img)
        return img #it will only return image because it is testing dataset and it don't have lable for it

testdataset=TestDataset('/kaggle/temp/test/')
testdataloader=DataLoader(dataset=testdataset, batch_size=batch_size)

In [ ]:
def eval(dataloader, model,loss_fn, path):
    model.eval()
    data=pd.read_csv(path) #data
    with torch.no_grad():#for efficiency purpose because if we don't want to backpropogate the why should we have to track of it
        for i, imgs in enumerate(dataloader): #i will be from 0 to 63 imgs will be 224 X 224 X 3
            finalbatchpred=np.zeros(imgs.shape[0],dtype='object') # imgs.shape[0] will be 64 here dtype is object because 
            #because altimetly we will be storing answer in word formate like "cat", "dog", "aeroplane" etc.
#             means  finalbatchpred will be 64 0's but type having as object 
            imgs=imgs.to(device)
            pred=model(imgs)#pred will be of 64 X 10
            
            pred=torch.argmax(pred,dim=1).type(torch.int).cpu() #now pred will be 64 X 1 and all 64 value will be from 0 to 9
            for j,p in enumerate(pred):
                finalbatchpred[j]=num2name[p.item()] #since p is tensor so p.item() will be in numeric value mean we are unpacking value from tensor
            data.iloc[i*batch_size:i*batch_size+batch_size ,1]=finalbatchpred #chanign default predicton ("cat") to predicted value
    
    data.to_csv('submission.csv', index=False) #so it will be stored into /kaggle/working/submission.csv
    data.head()

so first 50 prediction will be 
id,label
1,deer
2,deer
3,automobile
4,ship
5,airplane
6,cat
7,airplane
8,dog
9,ship
10,cat
11,bird
12,horse
13,frog
14,deer
15,dog
16,airplane
17,dog
18,bird
19,airplane
20,deer
21,ship
22,ship
23,ship
24,automobile
25,frog
26,truck
27,automobile
28,cat
29,cat
30,cat
31,cat
32,horse
33,dog
34,airplane
35,deer
36,cat
37,cat
38,horse
39,ship
40,airplane
41,bird
42,bird
43,frog
44,deer
45,horse
46,automobile
47,deer
48,bird
49,airplane

In [ ]:
eval(testdataloader, model,loss_fn, '/kaggle/input/cifar-10/sampleSubmission.csv')